#### See the [psf-weather-station demo](./psf-weather-station-demo.ipynb) notebook to learn more about usage of the package. Stay here for fun visualizations!

### setup and plotting things

Most users, in order to generate parameters and run simulations, do not need to replicate most of what is shown here. Explanations here are minimal -- more information is in the documentation.

In [1]:
import psfws
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
plt.rcParams['font.size']=14

In [3]:
seed = 9283456
ws = psfws.ParameterGenerator(seed=seed)

pt = ws._rng.choice(ws.data_fa.index)

In [4]:
def plot_params(params, j_gl, j_fa, j_cn2, h_profile, p_curves, fa_cn2, fa_h_km):
    f, a = plt.subplots(3, 1, figsize=(6,6), sharex=True, gridspec_kw={'hspace':0.2})
    
    # wind speed
    a[0].plot(params['h'], params['speed'], 'o', ms=10, color='#D52D00')
    a[0].set_ylabel(r'$|v|$ [m/s]')
    a[0].set_ylim([0, 60])
    
    # wind direction
    a[1].plot(params['h'], [d % 360 for d in params['phi']], 'o', ms=10, color='#D52D00')
    a[1].set_ylim(0, 360, 0)
    a[1].set_yticks([0, 90, 180, 270])
    a[1].set_ylabel(r'$\phi$ [$^\circ$]')
    
    # rescale turbulence integral with saved values
    j = [j_gl] + [j * j_fa / np.sum(params['j'][1:]) for j in params['j'][1:]]
    a[2].set_ylabel(r'$J$ [m$^{1/3}$]');
    a[2].plot(params['h'], j, 'o', ms=10, color='#D52D00', zorder=3)
    a[2].set_ylim([0, np.max([j_fa, j_gl]) + 1.5e-14])
    a[-1].set_xlabel('altitude [km]')
    
    # plotting the curves from which points are drawn:
    a[0].plot(h_profile, p_curves['speed'], color='#D52D00', alpha=.6, lw=2, zorder=2)
    a[1].plot(h_profile, np.array(p_curves['phi']) % 360, color='#D52D00', alpha=.6, lw=2, zorder=2)

    a[2].patch.set_visible(False)
    a2c = a[2].twinx()
    a2c.plot(fa_h_km, fa_cn2 * j_fa / j_cn2, color='#EF7627')    
    a2c.set_ylabel(r'$C_n^2(h)$ [$m^{-2/3}$]')
    a2c.set_zorder(a[2].get_zorder()-1)

    # show layer edges (just in this if statement for line ordering purposes)
    [[ax.axvline(e, color='lightgrey', lw=2, zorder=1) for e in params['edges'][1:]] for ax in [a[0], a[1], a2c]]
    [ax.axvline(params['edges'][0], color='lightgrey', linestyle='--', lw=2, zorder=1) for ax in [a[0], a[1], a2c]]

    plt.show();

### Choosing layer options

In [10]:
# getting raw measurements so we can interpolate + plot a smooth curve
measured = ws.get_measurements(pt)
h_profile = np.linspace(min(ws.h), max(ws.h), 500)
p_curves = ws._interpolate(measured, np.array(h_profile),s=0)

# fetching the cn2 profile and calculating the normalization
fa_cn2, fa_h_km = ws._get_fa_cn2(pt)
j_cn2 = psfws.utils.integrate_in_bins(fa_cn2, fa_h_km, [fa_h_km[0], fa_h_km[-1]])

# hack: we'll be calling "get_parameters" multiple times with the widget, so we are saving the J values
# here so that we don't draw different turbulence values for each function call while varying N.
j_gl, j_fa = ws._draw_j(pt=pt)

def plot_screens(n_screens, pt, location):
    params = ws.get_parameters(pt, nl=n_screens, location=location)
    plot_params(params, j_gl, j_fa, j_cn2, h_profile, p_curves, fa_cn2, fa_h_km)
    
widgets.interact(plot_screens, 
                 pt=widgets.fixed(pt), 
                 n_screens=widgets.IntSlider(min=2, max=14, step=1, value=8),
                 location=widgets.Dropdown(options=['com','mean'], value='mean', description='placement'));

interactive(children=(IntSlider(value=8, description='n_screens', max=14, min=2), Dropdown(description='placem…

Sliders dictate the number of phase screens and the placement of those screens, either using the mean (`mean`) of altitude or the center of mass (`com`) of $C_n^2$ with altitude.

Vertical grey lines denote the edges of the atmospheric layers, represented in the simulation by phase screens which are placed at the altitudes marked by red dots. The dashed grey line is the altitude of the telescope/ground. 

Note that in the top two panels, parameters are sampled *from* the curves, whereas the J values (reminder: this is the "turbulence integral", related to seeing) in the bottom panel are the integrals, within each layer, of the orange curve.

### Time variation
The plot above shows a snapshot in time of the atmospheric environment. How does this change with time? We can explore this by making a time slider as opposed to layer sliders, as done below. We've fixed `n_screens=8` and chosen the `com` screen location option.

In [7]:
def plot_time(date):
    # getting the raw measurements so we can interpolate a smooth curve to plot
    measured = ws.get_measurements(date)
    h_profile = np.linspace(min(ws.h), max(ws.h), 500)
    p_curves = ws._interpolate(measured, np.array(h_profile),s=0)
    
    # fetch cn2 profile and calculate normalization
    fa_cn2, fa_h_km = ws._get_fa_cn2(date)
    j_cn2 = psfws.utils.integrate_in_bins(fa_cn2, fa_h_km, [fa_h_km[0], fa_h_km[-1]])

    params = ws.get_parameters(date, nl=6, location='com')
    
    j_gl = params['j'][0]
    j_fa = np.sum(params['j'][1:])
    
    plot_params(params, j_gl, j_fa, j_cn2, h_profile, p_curves, fa_cn2, fa_h_km)
    
dates = ws.data_gl.index[0:50]    
widgets.interact(plot_time, 
                 date=widgets.SelectionSlider(options=dates, description='Date/Time'));

interactive(children=(SelectionSlider(description='Date/Time', options=(Timestamp('2019-05-01 00:00:00+0000', …